출처-메타코드m

# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ad = pd.read_csv("/content/drive/MyDrive/메타코드/데이터분석/ad1.csv")

In [4]:
# 광고관련 데이터
# date - 광고시작하는 날짜
# campaign - 광고이름
# ad_group - 캠페인 하위 그룹
# creative - 그룹 하위 개별 구분자
# spend - 광고에 들어간 돈

ad.head()


,date,campaign,ad_group,creative,spend,clicks
0,2023-01-01,Campaign_1,AdGroup_1,Creative_1,237.659902,22
1,2023-01-01,Campaign_1,AdGroup_1,Creative_2,469.650812,19
2,2023-01-01,Campaign_1,AdGroup_1,Creative_3,186.049658,89
3,2023-01-01,Campaign_1,AdGroup_2,Creative_1,156.240040,11
4,2023-01-01,Campaign_1,AdGroup_2,Creative_2,133.817095,16


In [7]:
ad.campaign.value_counts()

,count
campaign,
Campaign_1,531
Campaign_2,531
Campaign_3,531


In [9]:
ad[ad.campaign == "Campaign_1"].groupby( by = ['campaign', 'ad_group', 'creative']).size()

campaign    ad_group   creative  
Campaign_1  AdGroup_1  Creative_1    59
                       Creative_2    59
                       Creative_3    59
            AdGroup_2  Creative_1    59
                       Creative_2    59
                       Creative_3    59
            AdGroup_3  Creative_1    59
                       Creative_2    59
                       Creative_3    59
dtype: int64

In [10]:
log = pd.read_csv("/content/drive/MyDrive/메타코드/데이터분석/log1.csv")

In [11]:
# 기록 데이터
log.head()

,order_id,campaign,timestamp,event
0,1174,Campaign_3,2023-02-23 18:15:58,purchase
1,1491,NaN,2023-02-17 13:19:47,purchase
2,1163,Campaign_2,2023-01-08 09:05:28,purchase
3,1263,Campaign_1,2023-01-26 20:18:05,purchase
4,1169,Campaign_2,2023-02-12 22:49:20,purchase


In [12]:
od = pd.read_csv("/content/drive/MyDrive/메타코드/데이터분석/order1.csv")

In [13]:
od.head()

,order_id,order_date,order_amount
0,1,2023-02-07,663.424911
1,2,2023-02-13,1373.031257
2,3,2023-01-13,714.409280
3,4,2023-01-09,217.473674
4,5,2023-01-10,1235.813302


# 마케팅 데이터 전처리 자동화

- 주단위로 유입경로별 성과를 트레킹

### CPC : Cost Per Click 클릭당 얼마를 지불
### ROAS (Return On Advertising Spend)

- **정의**: 광고 지출에 대한 수익을 나타내는 마케팅 지표.

- **계산 방법**:
$$
\text{ROAS} = \frac{\text{광고로 인한 수익}}{\text{광고 지출}}
$$

- **중요성**:
  - **높은 ROAS**: 광고 지출 대비 높은 수익을 의미. 광고 캠페인의 효과성을 나타냄.
  - **낮은 ROAS**: 광고 투자에 대한 낮은 수익을 의미. 광고 전략의 개선이 필요함을 시사.


# 주별로 광고 데이터 집게

In [25]:
#  데이터 타입 변경
ad['date'] = pd.to_datetime(ad.date)
# 시작하는 날짜로 변경
ad['week'] = ad.date.dt.to_period('W').dt.to_timestamp()
ad.info()
ad.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593 entries, 0 to 1592
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      1593 non-null   datetime64[ns]
 1   campaign  1593 non-null   object        
 2   ad_group  1593 non-null   object        
 3   creative  1593 non-null   object        
 4   spend     1593 non-null   float64       
 5   clicks    1593 non-null   int64         
 6   week      1593 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 87.2+ KB


,date,campaign,ad_group,creative,spend,clicks,week
1588,2023-02-28,Campaign_3,AdGroup_2,Creative_2,107.936125,69,2023-02-27
1589,2023-02-28,Campaign_3,AdGroup_2,Creative_3,202.769327,33,2023-02-27
1590,2023-02-28,Campaign_3,AdGroup_3,Creative_1,101.871694,27,2023-02-27
1591,2023-02-28,Campaign_3,AdGroup_3,Creative_2,295.447590,65,2023-02-27
1592,2023-02-28,Campaign_3,AdGroup_3,Creative_3,145.928267,19,2023-02-27


In [28]:
ad_agg = ad.groupby(by=['campaign', 'week'])[["spend", 'clicks']].sum().reset_index()

In [30]:
ad_agg.head()
ad_agg['cpc'] = ad_agg.spend/ad_agg.clicks
ad_agg.head()

,campaign,week,spend,clicks,cpc
0,Campaign_1,2022-12-26,2241.418708,313,7.161082
1,Campaign_1,2023-01-02,14602.644897,3517,4.152017
2,Campaign_1,2023-01-09,18568.019047,3389,5.478908
3,Campaign_1,2023-01-16,16614.755062,3677,4.518563
4,Campaign_1,2023-01-23,16701.262833,3384,4.935361


# order 에 log 에서 유입경로 가져오기

In [33]:

log.timestamp = pd.to_datetime(log.timestamp)
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   order_id   1001 non-null   int64         
 1   campaign   474 non-null    object        
 2   timestamp  1001 non-null   datetime64[ns]
 3   event      1001 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 31.4+ KB


In [35]:
# 캠페인 정보 있는 것들만 필터일
log2 = log[~log.campaign.isna()]

In [37]:
od.order_date = pd.to_datetime(od.order_date)
od.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      2000 non-null   int64         
 1   order_date    2000 non-null   datetime64[ns]
 2   order_amount  2000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 47.0 KB


In [ ]:
#  중복되는 데이터가 있는지 체크
log.groupby('order_id').size().sort_values()
od.groupby('order_id').size().sort_values()

In [42]:
#  order_id가 교집합 데이터이기 때문에 합친다.
# 갖고있는 column이 아니라면 NaN 값이 대신 들어간다.

od1 = pd.merge(od, log, how='left', left_on='order_id', right_on='order_id')
# 캠페인으로 들어온 경우가 아닌 데이터는 organic으로 간주
# sns으로 들어온건 organic으로 보지 않음
# organic은 품질이 좋은 데이터임 본인 스스로 들어오기 때문...인가?
od1.head()

,order_id,order_date,order_amount,campaign,timestamp,event
0,1,2023-02-07,663.424911,NaN,NaT,NaN
1,2,2023-02-13,1373.031257,NaN,NaT,NaN
2,3,2023-01-13,714.409280,NaN,NaT,NaN
3,4,2023-01-09,217.473674,NaN,NaT,NaN
4,5,2023-01-10,1235.813302,NaN,NaT,NaN


In [43]:
od1.campaign = od1.campaign.fillna('organic')
od1.groupby(by = 'campaign').order_amount.sum()

,order_amount
campaign,
Campaign_1,2.161780e+05
Campaign_2,2.144346e+05
Campaign_3,9.549273e+04
organic,1.668108e+06


In [47]:
od1['week'] = od1.order_date.dt.to_period("W").dt.to_timestamp()

In [51]:
od_agg = od1.groupby(by=['campaign', 'week'])['order_amount'].sum().reset_index()

# 광고지출과 매출데이터 병합하기

In [52]:
ad_agg.head()

,campaign,week,spend,clicks,cpc
0,Campaign_1,2022-12-26,2241.418708,313,7.161082
1,Campaign_1,2023-01-02,14602.644897,3517,4.152017
2,Campaign_1,2023-01-09,18568.019047,3389,5.478908
3,Campaign_1,2023-01-16,16614.755062,3677,4.518563
4,Campaign_1,2023-01-23,16701.262833,3384,4.935361


In [54]:
od_agg.head()

,campaign,week,order_amount
0,Campaign_1,2022-12-26,5888.163592
1,Campaign_1,2023-01-02,29067.678837
2,Campaign_1,2023-01-09,27335.893357
3,Campaign_1,2023-01-16,23543.017742
4,Campaign_1,2023-01-23,24237.367534


In [56]:
df = pd.merge(od_agg, ad_agg, how="left", left_on = ['campaign', 'week'], right_on=['campaign', 'week'])
df.head()

,campaign,week,order_amount,spend,clicks,cpc
0,Campaign_1,2022-12-26,5888.163592,2241.418708,313.0,7.161082
1,Campaign_1,2023-01-02,29067.678837,14602.644897,3517.0,4.152017
2,Campaign_1,2023-01-09,27335.893357,18568.019047,3389.0,5.478908
3,Campaign_1,2023-01-16,23543.017742,16614.755062,3677.0,4.518563
4,Campaign_1,2023-01-23,24237.367534,16701.262833,3384.0,4.935361


In [61]:
# 클수록 수익이 많다

df['roas'] = df.order_amount / df.spend

In [62]:
df.head()

,campaign,week,order_amount,spend,clicks,cpc,roas
0,Campaign_1,2022-12-26,5888.163592,2241.418708,313.0,7.161082,2.626981
1,Campaign_1,2023-01-02,29067.678837,14602.644897,3517.0,4.152017,1.990576
2,Campaign_1,2023-01-09,27335.893357,18568.019047,3389.0,5.478908,1.472203
3,Campaign_1,2023-01-16,23543.017742,16614.755062,3677.0,4.518563,1.416995
4,Campaign_1,2023-01-23,24237.367534,16701.262833,3384.0,4.935361,1.451230


In [64]:
df.pivot_table(
    index="campaign",
    columns = "week",
    values = 'roas'
)

week,2022-12-26,2023-01-02,2023-01-09,2023-01-16,2023-01-23,2023-01-30,2023-02-06,2023-02-13,2023-02-20,2023-02-27
campaign,,,,,,,,,,
Campaign_1,2.626981,1.990576,1.472203,1.416995,1.451230,1.809280,1.190942,1.118135,1.579390,1.217456
Campaign_2,0.850365,1.180272,1.377531,1.607622,1.483889,2.071922,2.053962,1.044400,1.482987,1.098304
Campaign_3,0.620534,0.689514,0.512379,0.352103,0.548467,0.756456,0.730515,0.513122,0.905600,1.206279
